In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a4c52fb570f497dfd288e6feded63a35c11d8f230190e329ec729e82e3ee092a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

import pandas as pd

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
qa_model = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/FYP - Personal Folder/Multiprocessors.xlsx')
dataset = df.values

In [ ]:
embeddingsOfTheSubTopics = model.encode(dataset[:, 2])

In [ ]:
question = 'does cache listen to the snoop bus ?'
ecodedQuestion = model.encode(question)

In [ ]:
cos_similarities = util.cos_sim(embeddingsOfTheSubTopics, ecodedQuestion)

In [ ]:
cos_similarities

tensor([[0.1148],
        [0.2105],
        [0.3830],
        [0.3939],
        [0.6472],
        [0.3616],
        [0.1709],
        [0.2324],
        [0.3177],
        [0.1792],
        [0.1740]])

In [ ]:
def getTheRelevantRow(question):
  max = cos_similarities[0]
  maxIndex = 0
  for idx, score in enumerate(cos_similarities):
    if (score > max):
      max = score
      maxIndex = idx

  return max, maxIndex

In [ ]:
selectedSubTopicRow = getTheRelevantRow(question)[1]
selectedFileName = dataset[selectedSubTopicRow][3]
print(selectedFileName)

Multiprocessors_Bus_snooping


In [ ]:
def getRelevantPassage(filename):
    with open('/content/drive/MyDrive/FYP - Personal Folder/Files/' + filename + '.txt', encoding="utf8") as r:
        lines = r.readlines()
        return lines

In [ ]:
selectedPassage = getRelevantPassage(selectedFileName)[0]
selectedPassage

'\ufeffDedicated bus for coherency control. At every write access, cache controllers share addresses through the snoop bus. Each cache listens to the snoop bus,to check “if other caches are updating blocks that I also have cached”. Write-invalidate protocols:invalidate own copyNext access to same block is a miss!. Write-through caching: memory has up-to-date copy. Write-back caching: snoop in other caches to find most recent copy(more complex snooping, more snoop traffic, less memory traffic). Write-update protocols:update own copy (a.k.a. write-broadcast)Next access to same block is a hitNeed to share data+addressin snoop bus for every write(Increased snoop bus lines more power, less cache misses)'

In [ ]:
def answerForTheQuestion(question):
  print(question)
  print(qa_model(question=question, context=selectedPassage))

In [ ]:
answerForTheQuestion(question)

does cache listen to the snoop bus ?
{'score': 0.0663423165678978, 'start': 118, 'end': 153, 'answer': 'Each cache listens to the snoop bus'}


In [ ]:
from transformers import pipeline

oracle = pipeline(model="deepset/roberta-base-squad2")
def answerForTheQuestionFromRoberta(question):
  print(question)
  print(oracle(question=question, context=selectedPassage))

In [ ]:
answerForTheQuestionFromRoberta(question)

does cache listen to the snoop bus ?
{'score': 0.26040855050086975, 'start': 118, 'end': 153, 'answer': 'Each cache listens to the snoop bus'}


In [ ]:
from transformers import pipeline

# this is a light model
oracle = pipeline(model="twmkn9/bert-base-uncased-squad2")
def answerForTheQuestionFromUncased(question):
  print(question)
  print(oracle(question=question, context=selectedPassage))

answerForTheQuestionFromUncased(question)

Some weights of the model checkpoint at twmkn9/bert-base-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


does cache listen to the snoop bus ?
{'score': 0.7049338221549988, 'start': 118, 'end': 136, 'answer': 'Each cache listens'}


In [ ]:
from transformers import pipeline

oracle = pipeline(model="distilbert-base-cased-distilled-squad")
def answerForTheQuestionFromDistilled(question):
  print(question)
  print(oracle(question=question, context=selectedPassage))

answerForTheQuestionFromDistilled(question)

does cache listen to the snoop bus ?
{'score': 0.0663423165678978, 'start': 118, 'end': 153, 'answer': 'Each cache listens to the snoop bus'}


In [ ]:
from transformers import pipeline

oracle = pipeline(model="bert-large-uncased-whole-word-masking-finetuned-squad")
def answerForTheQuestionFromFinetunedSqaud(question):
  print(question)
  print(oracle(question=question, context=selectedPassage))

answerForTheQuestionFromFinetunedSqaud(question)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


does cache listen to the snoop bus ?
{'score': 0.21974915266036987, 'start': 118, 'end': 153, 'answer': 'Each cache listens to the snoop bus'}


In [ ]:
from transformers import pipeline

context =  "More about interrupts\nThe ability to handle interrupts and exceptions is an important feature for processors.\nWe have added the control logic to detect the two types of exceptions described earlier, but note that the Cause and the EPC register cannot be read.\nInstructions would have to be provided to allow these registers to be read and manipulated.\nProcessors usually have policies relating to exceptions. The MIPS processor had the policy that instructions which cause an exception has no effect (e.g., nothing is written into a register.)\nFor some exceptions, if this policy is used, the operation may have to complete before the exception can be detected, and the result of the operation must then be “rolled back.”\nThis makes the implementation of exceptions difficult — sometimes the state prior to an operation must be saved so it can be restored.\nThis constraint alone sometimes results in instructions requiring more cycles for their implementation.\n299\nExceptions and interrupts in other processors\nA common type of interrupt is a vectored interrupt. Here, different interrupts or exceptions jump to different addresses in memory.\nThe operating system places an appropriate interrupt handler for the particular interrupt at each of these locations.\nA vectored interrupt both identifies the type of interrupt, and provides the handler at the same time. (Since different interrupts or exceptions have different vectors.)\nIn the INTEL processors, it is the responsibility of the interrupting device to provide the interrupt vector. (This is usually done by one of the peripheral controller chips, under control of the operating system.)\nA major problem with the PC architecture is that only a small number of interrupts (typically 16) can be handled by the controller chip.\nthis has lead to many problems with hardware devices “sharing interrupts” — defeating the advantages of vectored interrupts.\nWe will look at interrupts again, later, when we discuss input and output devices."
question = "What is a vectored interrupt, and how does it work?"

# this is a light model
oracle = pipeline(model="twmkn9/bert-base-uncased-squad2")
print(oracle(question=question, context=context))


Some weights of the model checkpoint at twmkn9/bert-base-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'score': 0.32008060812950134, 'start': 1282, 'end': 1362, 'answer': 'both identifies the type of interrupt, and provides the handler at the same time'}
